<a href="https://colab.research.google.com/github/Jayshree2816/Music-Genres/blob/master/Music_Genres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [ ]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
from glob import glob

In [ ]:
!pip install memory_profiler

  Created wheel for memory-profiler: filename=memory_profiler-0.58.0-cp36-none-any.whl size=30181 sha256=80cdb9731fb60743942e6f570c18eaabb3196efbe8fbc21f42ac07917a6057fd
  Stored in directory: /root/.cache/pip/wheels/02/e4/0b/aaab481fc5dd2a4ea59e78bc7231bb6aae7635ca7ee79f8ae5
Successfully built memory-profiler


In [ ]:
%%capture
!apt-get install libav-tools -y

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
import librosa
import librosa.display
import pylab
from matplotlib import figure
import gc
from path import Path
from sklearn.metrics import mean_squared_error

# Import Dataset

In [ ]:
!pip install path

In [ ]:
!wget https://www.dropbox.com/s/4jw31k5mlzcmgis/genres.tar.gz

--2021-02-12 04:12:09--  https://www.dropbox.com/s/4jw31k5mlzcmgis/genres.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/4jw31k5mlzcmgis/genres.tar.gz [following]
--2021-02-12 04:12:09--  https://www.dropbox.com/s/raw/4jw31k5mlzcmgis/genres.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd2442edfdd0f8c5f9e7ead4559.dl.dropboxusercontent.com/cd/0/inline/BIxEuaP_-NkEIOE7ID0EUM_3jKomdWV1ufSR0CB2z4Nu0LeNDEB884onqCAPkMQmx4crXLxYBwXS06dJenCmVgW6tMuDlsal4Xl9WTSYaMSFKA/file# [following]
--2021-02-12 04:12:09--  https://ucd2442edfdd0f8c5f9e7ead4559.dl.dropboxusercontent.com/cd/0/inline/BIxEuaP_-NkEIOE7ID0EUM_3jKomdWV1ufSR0CB2z4Nu0LeNDEB884onqCAPkMQmx4crXLxYBwXS06dJenCmVgW6tMuDlsal4Xl9WTSYaMSFKA/file
Resolving ucd

In [ ]:
import tarfile

fname = '/content/genres.tar.gz'

if fname.endswith("tar.gz"):

    tar = tarfile.open(fname, "r:gz")
    tar.extractall()
    tar.close()

In [ ]:
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt
import pathlib
#Keras
import keras

import warnings
warnings.filterwarnings('ignore')

In [ ]:
cmap = plt.get_cmap('inferno')

plt.figure(figsize=(10,10))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    pathlib.Path(f'/content/ouput/{g}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'/content/genres/{g}'):
        songname = f'/content/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'/content/ouput/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

<Figure size 720x720 with 0 Axes>

In [ ]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [ ]:
file = open('music_data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'/content/genres/{g}'):
        songname = f'/content/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        rmse = librosa.feature.rms(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('music_data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [ ]:
music_data = pd.read_csv("music_data.csv")

In [ ]:
music_data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00059.au,0.405961,0.169987,2235.906951,2100.004335,4390.031787,0.142288,-86.225304,91.948448,-23.408659,68.253288,-3.895873,22.745262,-9.899613,16.895977,-11.397531,18.431677,-7.015263,10.771054,-11.938685,1.005814,-5.923496,5.318165,-6.140526,1.948991,-5.102755,-0.666248,blues
1,blues.00048.au,0.379428,0.123986,2148.786199,2204.574064,4673.663406,0.098812,-106.311028,102.630951,-20.155123,59.360367,-9.954543,33.689182,-14.170398,23.509998,-13.314068,16.014284,-18.916981,7.821362,-7.788121,4.047834,-11.340939,2.060218,-5.739425,0.464256,-1.909088,-3.128308,blues
2,blues.00000.au,0.349943,0.130225,1784.420446,2002.650192,3806.485316,0.083066,-113.596748,121.557297,-19.158825,42.351032,-6.376458,18.618876,-13.697912,15.344631,-12.285267,10.980492,-8.324325,8.810669,-3.667368,5.751691,-5.162763,0.750948,-1.691938,-0.409953,-2.300209,1.219929,blues
3,blues.00033.au,0.260669,0.133165,1430.582003,2075.602343,3246.730367,0.041970,-239.010910,108.669212,34.149349,30.242792,-0.470996,27.296513,-21.389084,14.155845,-24.505630,-1.876820,-28.052265,-11.258264,-16.484100,-7.636357,-12.796290,-15.352031,-13.732283,-7.400161,-10.083954,-13.285140,blues
4,blues.00068.au,0.289932,0.103115,2513.716817,2345.230614,5247.443269,0.135146,-113.167763,88.709335,-8.925343,49.487495,-34.580006,15.865089,-17.744377,14.714100,-11.207615,15.366563,-15.081731,8.748788,-18.630899,5.474217,-9.267430,8.017963,-9.552074,5.873594,-4.402835,3.615348,blues


In [ ]:
music_data.shape

(1000, 28)

In [ ]:
music_data = music_data.drop(['filename'],axis=1)

In [ ]:
genre_list = music_data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(music_data.iloc[:, :-1], dtype = float))

## **Split Data into Train and Test dataset**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
len(y_train)

800

In [ ]:
len(y_test)

200

In [ ]:
X_train[10]

array([ 1.01433865,  0.81671353,  0.35988841, -0.04020464,  0.28746135,
        0.9376157 ,  1.28698228,  0.01198026, -1.31429583,  0.57113261,
       -1.59277563,  0.91571936, -1.34385334,  1.55195153, -1.57117203,
        1.66908281, -1.09651671,  0.94660548, -1.71697634,  0.35046924,
       -0.75989675,  0.57876513, -0.88358627,  1.63409889,  0.10597742,
        1.27175015])

## **Model Selection**

In [ ]:
from keras import models
from keras import layers
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)

Epoch 1/20
7/7 [==============================] - 1s 4ms/step - loss: 2.2221 - accuracy: 0.1909
Epoch 2/20
7/7 [==============================] - 0s 4ms/step - loss: 1.8716 - accuracy: 0.3499
Epoch 3/20
7/7 [==============================] - 0s 4ms/step - loss: 1.6497 - accuracy: 0.3914
Epoch 4/20
7/7 [==============================] - 0s 5ms/step - loss: 1.4915 - accuracy: 0.4630
Epoch 5/20
7/7 [==============================] - 0s 3ms/step - loss: 1.3410 - accuracy: 0.5078
Epoch 6/20
7/7 [==============================] - 0s 3ms/step - loss: 1.2373 - accuracy: 0.5878
Epoch 7/20
7/7 [==============================] - 0s 3ms/step - loss: 1.1145 - accuracy: 0.6607
Epoch 8/20
7/7 [==============================] - 0s 3ms/step - loss: 1.0317 - accuracy: 0.6725
Epoch 9/20
7/7 [==============================] - 0s 3ms/step - loss: 0.9492 - accuracy: 0.6983
Epoch 10/20
7/7 [==============================] - 0s 3ms/step - loss: 0.9199 - accuracy: 0.6975
Epoch 11/20
7/7 [======================

In [ ]:
test_loss, test_acc = model.evaluate(X_test,y_test)

7/7 [==============================] - 0s 2ms/step - loss: 0.9580 - accuracy: 0.6550


In [ ]:
print('test_acc: ',test_acc)

test_acc:  0.6549999713897705


In [ ]:
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

In [ ]:
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=30,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)

Epoch 1/30
2/2 [==============================] - 1s 239ms/step - loss: 2.3227 - accuracy: 0.1189 - val_loss: 2.1546 - val_accuracy: 0.3200
Epoch 2/30
2/2 [==============================] - 0s 49ms/step - loss: 2.1483 - accuracy: 0.3103 - val_loss: 1.9910 - val_accuracy: 0.4150
Epoch 3/30
2/2 [==============================] - 0s 49ms/step - loss: 2.0035 - accuracy: 0.3734 - val_loss: 1.8224 - val_accuracy: 0.4200
Epoch 4/30
2/2 [==============================] - 0s 48ms/step - loss: 1.8654 - accuracy: 0.3713 - val_loss: 1.6728 - val_accuracy: 0.4100
Epoch 5/30
2/2 [==============================] - 0s 47ms/step - loss: 1.7338 - accuracy: 0.3883 - val_loss: 1.5617 - val_accuracy: 0.4500
Epoch 6/30
2/2 [==============================] - 0s 48ms/step - loss: 1.6159 - accuracy: 0.4211 - val_loss: 1.4793 - val_accuracy: 0.4850
Epoch 7/30
2/2 [==============================] - 0s 51ms/step - loss: 1.5009 - accuracy: 0.4802 - val_loss: 1.4100 - val_accuracy: 0.5300
Epoch 8/30
2/2 [==========

In [ ]:
results

[1.0143439769744873, 0.675000011920929]

## **Predict Result on Test dataset**

In [ ]:
predictions = model.predict(X_test)

In [ ]:
predictions[0].shape

(10,)

In [ ]:
np.sum(predictions[0])

0.99999994

In [ ]:
np.argmax(predictions[0])

0

In [ ]:
test_pred = model.predict_classes(X_test)
test_proba = model.predict_proba(X_test)
test_proba

array([[5.8554071e-01, 8.6947321e-04, 1.0828886e-01, ..., 1.7678434e-03,
        5.9358697e-02, 1.2666892e-01],
       [7.5759039e-05, 9.8157364e-01, 3.6106113e-04, ..., 3.2491582e-05,
        2.7662786e-04, 4.7426496e-04],
       [1.1461085e-01, 7.9797421e-05, 7.4862581e-01, ..., 9.4212429e-04,
        1.1424050e-02, 9.8812595e-02],
       ...,
       [2.1909881e-01, 1.3619431e-05, 1.6622326e-01, ..., 1.2726654e-04,
        9.6479254e-03, 5.7380271e-01],
       [1.6883024e-03, 1.5385125e-06, 1.1300856e-03, ..., 5.5736564e-06,
        1.8104595e-01, 2.6906973e-03],
       [3.4849395e-03, 2.8543421e-03, 2.2076277e-01, ..., 2.1200219e-01,
        3.1062126e-02, 1.4662406e-01]], dtype=float32)

In [ ]:
accuracy_score(y_test, test_pred)

0.675